## Object Detection + VLMs(Visual Language Models)

### Project Overview

⸻

**Problems &  Motivation**
1. Closed-set | 모델을 사전에 정의 + 학습된 제한된 객체만 인식 가능

2. Retraining | 새로운 객체 or 새로운 환경에 모델을 적용하는 경우 <br> dataset rebuild, retraining, finetuning 필요

<br> 


**New approch**

<br> object detection + agentic - 시간↓ 비용↓ 정확도↑

1. **Open-Vocabulary**
<br>
    * Multi modal architecture | Transformer 기반 CLIP, 텍스트와 이미지의 공유 임베딩 공간 학습<br>
    훈련 데이터에 없던 새로운 클래나 문구를 이해 + 이미지의 시각적 특징과 연결<br>
    GroundingDINO, OWL-ViT 와 같은 모델 사용

<br>

2. **Agentic Framework**
<br>

    * CLIP 기반 object detection process(GroundingDINO, OWL-ViT ) +  LLM(GPT, Gemini etc.. )
    * **한 번의 추론 과정** 내에서 검증과 비평 과정 수행
    * 깊게 생각하게 만들어 재학습 없이도 정확도 ↑
    * 교차 검증 방식을 상황/ 조건에 맞게 **선택, 조합, 반복 그리고 중지**할 수 있는 구조  
    → Agentic 성격 부여
    * 사용자의 의도에 부합하는 결과 출력

<img src="../notebook/figure/workflow_image.jpg" width="80%">

**VLM 호출 최적화**
<br> 검증의 방식으로 가능한 경우의 수

1. 개별 검증 | Brute-force crop loop 
    * 직관적인 방법 
    * 생성된 모든 바운딩 박스를 각각 독립적으로 처리<br>즉, n개의 bbox = n번의 처리
    * 바운딩 박스 영역만 잘라내기<br>→ base64 형식으로 인코딩<br>→ VLM에 질문할 프롬프트 생성 <br>→VLM API 분석<br>→VLM의 답변에 따라 제거 또는 유지
    * 시간↑ 비용↑

2. Numbered Batching 
    * Numbered-arrow annotation
    * 모든 detection 결과에 번호와 화살표를 단 이미지 한 장을 VLM으로 전달
    * 시간↓ 비용↓ 정확도↑

**High Level Design**
1. **Concept Detector**
    <br>image + user request <br>→ VLM(a)이 이미지와 사용자 요청을 분석해 detection할 객체의 개념 인식 +  object labels 추출

2. **Initial Object Detection**
    <br>image + object labels <br>→ Open-Vocabulary Object Detector를 사용, 이미지 내 bbox 생성

3. Visualization
    <br>image + initial detection <br>→ 탐지한 객체를 원본 이미지에 화살표, 숫자 라벨로 표시한 annotated image 생성

4. Query Critique & Refinement
    <br>annotated image + user request <br>→ VLM(b) 이 초기 탐지 결과 검토, <br>탐지 결과가 사용자의 의도와 맞지 않거나 부적절한 경우 더 일반적이거나 적합한 개념으로 object label 수정

5. Refined Detection
    <br>image + 개선한 object labels <br>→ Open-Vocabulary Object Detector를 사용, 이미지 내 bbox 생성

6. Final Filtering
    <br>image + user request + detection <br>→ VLM(a) 교차 검증<br>개선된 탐지 결과가 사용자의 의도와 일치하는지 검증, 사용자의 의도와 불일치 or 무관한 객체 제거  

7. Outut packaging
    <br>최종 검증된 bbox가 표시된 이미지
    <br>모든 검증 단계를 통과해 사용자 요청과 가장 일치하는 객체만 표시

<img src="../notebook/figure/workflow.jpg" width="80%">

<img src = "../notebook/figure/workflow_image.jpg" width = "80%">

<ime src="notebook\figure\workflow_image.jpg" width="80%">

**설계 계획**


1. VLM | VLM과의 연결
    * 프롬프트 구성, API 호출, 응답 파싱 
    * 메서드
        * init: OpenAI 클라이언트 초기화  
        * chat completion: VLM API 요청, 응답 저장
        * extract objects bfrom request: 탐지 객체 목록 문자열 리스트로 추출

2. Detecion | agentic pipeline 전체
    * 메서드
        * init: detection model processor 로드, VLM 인스턴스 주입
        * run: 전체 파이프라인 실행, 최종 결과 이미지 반환 
    * 내부 메서드        
        * critique, refine: 사용자의 요청에 부합하는지 비평, 개선된 객체 목록 추출
        * validate bbox num: 번호가 매겨진 탐지 이미지에서 사용자 의도와 일치하는 객체 번호 목록 추출
        * run detector: detetion 모델을 직접 실행해 이미지와 객체 리스에서 바운딩 박스 좌표 추출
    

3. Utils | 반복 사용 함수
    * image encoding
    * draw arrow and numberin
    * draw bounding box 

4. Parameter configuration | 기본 설정
    * model
        * detector model type: hugging face ID
        * initial VLM: 초기 개념 추출에 사용할 VLM 모델 이름
        * critique VLM: 쿼리 비평 및 개선에 사용할 VLM 모델 이름
        * validation VLM: 최종 검증에 사용할 VLM 모델 이름
    * execution configuration 
        * 코드 실행 장치: cuda or cpu
        * confidence threshold
    * visualization configuration: detection의 신뢰도
        * color palette: 바운딩 박스 및 라벨 시각화 
        * font path: 라벨 텍스트 그릴 때 사용할 폰트 파일 경로